In [2]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
from IPython.display import display
import random
pd.options.display.max_columns = None

test_obs = 401590
train_obs = 1708338

In [ ]:
%%time
from ast import literal_eval

def parse(csv_path='/Volumes/My Passport/Kaggle Competition/train_sample.csv', nrows=None):

    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    df['hits']=df['hits'].apply(literal_eval)
    df['hits']=df['hits'].str[0]
    df=df[pd.notnull(df['hits'])]
    
    df['customDimensions']=df['customDimensions'].apply(literal_eval)
    df['customDimensions']=df['customDimensions'].str[0]
    df=df[pd.notnull(df['customDimensions'])]
    
    
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource','hits','customDimensions']
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    return df
    
def load_data(nrows=None):
    train_df = parse(nrows=nrows)
    #test_df = parse("/Volumes/My Passport/Kaggle Competition/train_v2.csv",nrows)
    return train_df#,test_df

df_train=load_data(nrows=10000)

In [ ]:
df_train.shape

In [ ]:
# make NANs as 0 and set all values as float
#df_train['totals.transactionRevenue'] = df_train['totals.transactionRevenue'].astype(float)
columns_to_float = ['visitNumber', 'totals.bounces', 'totals.sessionQualityDim', 'totals.hits', 'totals.newVisits', 'totals.pageviews','totals.timeOnSite','totals.totalTransactionRevenue', 'totals.transactionRevenue','totals.transactions','totals.visits']
for column in columns_to_float:
    df_train[column].fillna(0, inplace=True)
    df_train[column] = df_train[column].astype(float)

In [ ]:
#df_train['bought_something'] = df_train['totals.transactionRevenue'] > 0
df_train['bought_something'] = df_train['totals.transactionRevenue'].apply(lambda x: 1 if x > 0 else 0)
df_train['country_us'] = df_train['geoNetwork.country'].apply(lambda x: 1 if x == 'United States' else 0)

In [ ]:
# Gather date/time data
from datetime import datetime
format_str = '%Y%m%d'
df_train['formatted_date'] = df_train['date'].apply(lambda x: datetime.strptime(str(x), format_str))
df_train['month'] = df_train['formatted_date'].apply(lambda x:x.month)
df_train['weekday'] = df_train['formatted_date'].apply(lambda x:x.weekday()) #mon to sun; 0 to 6
df_train['visitStartHour'] = df_train['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))
df_train.drop(['date','visitStartTime'], axis=1, inplace=True)


In [ ]:
# group sources
to_replace = [r'.*google.*',r'.*yahoo.*',r'.*\.facebook.*',r'(\.com)',r'(ml\.)',r'(m\.)',] #to group all google ,facebook types
value = [r'google',r'yahoo',r'facebook',r'',r'',r'']
df_train['trafficSource.source'] = df_train['trafficSource.source'].replace(to_replace, value, regex=True)

In [ ]:
train_columns_to_keep = [
    'visitNumber',
    'device.isMobile',
    'totals.bounces',
    'totals.hits',
    'totals.newVisits',
    'totals.pageviews',
    'totals.sessionQualityDim',
    'totals.timeOnSite',
    'totals.visits',
    'trafficSource.medium',
    'country_us',
    'month',
    'totals.transactions',
    'totals.transactionRevenue',
    'totals.totalTransactionRevenue',
    'bought_something'
]
df_train_sub = df_train.filter(train_columns_to_keep , axis = 1)
df_train_sub.head()

In [ ]:
columns_to_dummies = [
    'trafficSource.medium',
]
df_train_dum = pd.get_dummies(df_train_sub, columns = columns_to_dummies, drop_first=True)
df_train_sub = df_train_dum.drop(['totals.transactions','totals.transactionRevenue','totals.totalTransactionRevenue','bought_something'],axis=1)
df_train_response = df_train_dum.filter(['totals.transactions','totals.transactionRevenue','totals.totalTransactionRevenue','bought_something'])

In [ ]:
bought_sub = df_train_dum[df_train_dum['bought_something'] == 1]
bought_sub_response = bought_sub.filter(['totals.transactions','totals.transactionRevenue','totals.totalTransactionRevenue','bought_something'])
bought_sub_regress = bought_sub.drop(['totals.transactions','totals.transactionRevenue','totals.totalTransactionRevenue','bought_something'],axis=1)

In [1]:
#import sklearn.linear_model
#log_reg = sklearn.linear_model.LogisticRegression()
#fit = log_reg.fit(df_train_sub, df_train_response['bought_something'])
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

#bought_predictions = fit.predict(df_train_sub)
#df_train_sub['bought_predictions'] = pred_series.values

In [ ]:
sum(df_train_sub['bought_predictions'])

In [ ]:
from sklearn.linear_model import LinearRegression
lin_fit = LinearRegression().fit(bought_sub_regress, bought_sub_response['totals.transactionRevenue'])

In [ ]:
list_of_predictions = []
for index, row in df_train_sub.iterrows():
    if row['bought_predictions'] == 1:
        list_of_values.append(lin_fit.predict([row[:-2]])[0])
    else:
        list_of_values.append(0)